In [468]:
import urllib.request, json
import pandas as pd
import numpy as np
import time

import et_analysis as et

pd.set_option('display.max_columns', None)

In [469]:
# GTIP verisinin 4 kodlu kırılımda yüklenmesi;

df_gtip = pd.read_excel("GTIP_4.xlsx")
df_gtip['HS4'] = df_gtip['HS4'].str.replace("'", '')

necs_cols = ["period", "reporterCode", "flowCode", "partnerCode", 
             "partner2Code", "cmdCode", "cifvalue", "fobvalue", "primaryValue"]

In [470]:
# Burada UNcomtrade API'sinden veri çekme işlemi gerçekleştiriyoruz.

df = pd.DataFrame()
for i in df_gtip.HS4:   
    url = f"https://comtradeapi.un.org/public/v1/preview/C/A/HS?reporterCode=792&period=2022&partnerCode=0&partner2Code=0&cmdCode={i}&flowCode=X&customsCode=C00&motCode=0"
    hdr ={
    # Request headers
    'Cache-Control': 'no-cache',
    }
    req = urllib.request.Request(url, headers=hdr)
    req.get_method = lambda: 'GET'
    response = urllib.request.urlopen(req)
  
    a = response.read()
    a = json.loads(a.decode('utf-8'))
  
    df_gt = pd.DataFrame(a["data"])
    df = pd.concat([df, df_gt], ignore_index=True)
    time.sleep(1)

In [471]:
# df[necs_cols].to_csv("GTIP_4_tr_export_2022.csv")

In [472]:
years = ['Product code', "2016", "2017", "2018", "2019", "2020", "2021"]
years_ = ["2016", "2017", "2018", "2019", "2020", "2021"]

In [473]:
# Dünya ticaret verisini GTIP4 kırılımında yükleyelim;

wo_df = pd.read_csv("gtip_4_world_to_world_exports.csv")
wo_df['Product code'] = wo_df['Product code'].str.replace("'", '')
wo_df[years].head()

,Product code,2016,2017,2018,2019,2020,2021
0,0101,2360432.0,2711826.0,2990785.0,3201458.0,2904259.0,3689448.0
1,0102,7971043.0,8513819.0,9736621.0,9298932.0,8746296.0,8931748.0
2,0104,2078555.0,1852167.0,1871188.0,1924345.0,1553544.0,1796801.0
3,0105,2899697.0,2953581.0,3251469.0,3180560.0,2965443.0,3310070.0
4,0106,1367436.0,1436805.0,1397486.0,1136136.0,1190544.0,1549566.0


In [474]:
# Türkiye'nin ihracat verisini GTIP4 kırılımında yükleyelim;

tr_df = pd.read_csv("gtip_4_tr_export_to_world.csv")
tr_df['Product code'] = tr_df['Product code'].str.replace("'", '')
tr_df[years].head()

,Product code,2016,2017,2018,2019,2020,2021
0,0101,168,230,85,246,179,209
1,0102,0,0,478,3534,4522,8197
2,0104,33,0,2702,28949,28928,37295
3,0105,24667,31021,51036,47993,46179,56165
4,0106,3106,3422,3666,3891,3599,4531


In [475]:
# İhracat verilerinin betimsel istetistiklerine bakalım; 
quantiles = [0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]
tr_df[years_].describe(quantiles).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
2016,1229.0,116034.387307,463472.669678,0.0,0.0,9.0,71.0,788.0,9455.0,58134.0,250626.4,472264.2,1625533.16,8356130.0
2017,1229.0,127740.382425,520094.417396,0.0,0.0,7.0,50.8,924.0,9807.0,60799.0,290444.2,526086.4,1800440.08,11814903.0
2018,1229.0,136634.561432,527499.555752,0.0,0.0,9.4,57.6,1013.0,11452.0,71189.0,307102.4,584481.4,1959593.32,12441971.0
2019,1229.0,147169.093572,555808.625441,0.0,0.0,13.0,92.8,1316.0,13842.0,80444.0,319230.4,643798.4,1997645.84,12096068.0
2020,1229.0,138045.511798,456091.646005,0.0,0.0,7.0,92.0,1299.0,13737.0,80946.0,339261.2,620042.2,1967859.84,9636838.0
2021,1229.0,183290.728234,587209.915945,0.0,0.0,20.4,153.4,1838.0,19350.0,116719.0,446696.0,784084.6,2676650.64,9573494.0


In [476]:
# İhracat verilerindeki, aykırı değerleri %2.5'luk kısımlarından tıraşlayalım;
for year in years_:
    print(year, et.check_outlier(tr_df, year, q1=0.025, q3=0.975))
    et.replace_with_thresholds(tr_df, year, q1=0.025, q3=0.975)
    print(year, et.check_outlier(tr_df, year, q1=0.025, q3=0.975))
    print("~~~~~~~~~~~~")

2016 True
2016 False
~~~~~~~~~~~~
2017 True
2017 False
~~~~~~~~~~~~
2018 True
2018 False
~~~~~~~~~~~~
2019 True
2019 False
~~~~~~~~~~~~
2020 True
2020 False
~~~~~~~~~~~~
2021 True
2021 False
~~~~~~~~~~~~


In [477]:
# Dünya ticaretin verilerindeki, aykırı değerleri %2.5'luk kısımlarından tıraşlayalım;
for year in years_:
    print(year, et.check_outlier(wo_df, year, q1=0.025, q3=0.975))
    et.replace_with_thresholds(wo_df, year, q1=0.025, q3=0.975)
    print(year, et.check_outlier(wo_df, year, q1=0.025, q3=0.975))
    print("~~~~~~~~~~~~")

2016 True
2016 False
~~~~~~~~~~~~
2017 True
2017 False
~~~~~~~~~~~~
2018 True
2018 False
~~~~~~~~~~~~
2019 True
2019 False
~~~~~~~~~~~~
2020 True
2020 False
~~~~~~~~~~~~
2021 True
2021 False
~~~~~~~~~~~~


In [478]:
# İlgili yılları filtreleyelim ve ilgili yıllar için RCA istatistiğinin "numerator" değerini hesaplayalım;
tr_df = tr_df[years]
for year in years_:
    tr_df[f"num_{year}"] = tr_df[year]/tr_df[year].sum()

tr_df

,Product code,2016,2017,2018,2019,2020,2021,num_2016,num_2017,num_2018,num_2019,num_2020,num_2021
0,0101,168.0,230.0,85.0,246.0,179.0,209.0,1.366413e-06,0.000002,5.679504e-07,0.000002,0.000001,1.007403e-06
1,0102,0.0,0.0,478.0,3534.0,4522.0,8197.0,0.000000e+00,0.000000,3.193886e-06,0.000022,0.000029,3.951043e-05
2,0104,33.0,0.0,2702.0,28949.0,28928.0,37295.0,2.684026e-07,0.000000,1.805414e-05,0.000181,0.000184,1.797660e-04
3,0105,24667.0,31021.0,51036.0,47993.0,46179.0,56165.0,2.006269e-04,0.000229,3.410108e-04,0.000300,0.000293,2.707214e-04
4,0106,3106.0,3422.0,3666.0,3891.0,3599.0,4531.0,2.526238e-05,0.000025,2.449537e-05,0.000024,0.000023,2.183991e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,9703,903.0,364.0,3383.0,847.0,333.0,1670.0,7.344471e-06,0.000003,2.260443e-05,0.000005,0.000002,8.049582e-06
1225,9704,0.0,0.0,0.0,0.0,0.0,21.0,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,1.012223e-07
1226,9705,355.0,310.0,204.0,1730.0,231.0,169.0,2.887361e-06,0.000002,1.363081e-06,0.000011,0.000001,8.145984e-07
1227,9706,0.0,0.0,56.0,0.0,0.0,4.0,0.000000e+00,0.000000,3.741791e-07,0.000000,0.000000,1.928044e-08


In [479]:
# İlgili yılları filtreleyelim ve ilgili yıllar için RCA istatistiğinin "denominator" değerini hesaplayalım;
wo_df = wo_df[years]
for year in years_:
    wo_df[f"denom_{year}"] = wo_df[year]/wo_df[year].sum()

wo_df

,Product code,2016,2017,2018,2019,2020,2021,denom_2016,denom_2017,denom_2018,denom_2019,denom_2020,denom_2021
0,0101,2.360432e+06,2.711826e+06,2.990785e+06,3.201458e+06,2.904259e+06,3.689448e+06,0.000183,0.000193,0.000195,0.000215,0.000206,0.000208
1,0102,7.971043e+06,8.513819e+06,9.736621e+06,9.298932e+06,8.746296e+06,8.931748e+06,0.000619,0.000605,0.000635,0.000623,0.000620,0.000504
2,0104,2.078555e+06,1.852167e+06,1.871188e+06,1.924345e+06,1.553544e+06,1.796801e+06,0.000161,0.000132,0.000122,0.000129,0.000110,0.000101
3,0105,2.899697e+06,2.953581e+06,3.251469e+06,3.180560e+06,2.965443e+06,3.310070e+06,0.000225,0.000210,0.000212,0.000213,0.000210,0.000187
4,0106,1.367436e+06,1.436805e+06,1.397486e+06,1.136136e+06,1.190544e+06,1.549566e+06,0.000106,0.000102,0.000091,0.000076,0.000084,0.000087
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,9703,4.764335e+06,4.573594e+06,4.673446e+06,5.206415e+06,2.625737e+06,4.374750e+06,0.000370,0.000325,0.000305,0.000349,0.000186,0.000247
1225,9704,1.682750e+05,1.256990e+05,1.362620e+05,1.129970e+05,7.731200e+04,1.016640e+05,0.000013,0.000009,0.000009,0.000008,0.000005,0.000006
1226,9705,1.165672e+06,1.275632e+06,1.220421e+06,1.464492e+06,1.206239e+06,1.813708e+06,0.000091,0.000091,0.000080,0.000098,0.000085,0.000102
1227,9706,2.685712e+06,2.485044e+06,2.789587e+06,3.236065e+06,1.922090e+06,2.102801e+06,0.000209,0.000176,0.000182,0.000217,0.000136,0.000119


In [480]:
tr_rca = pd.DataFrame(columns=["Product code", "2016", "2017", "2018", "2019", "2020", "2021"])
tr_rca["Product code"] = wo_df["Product code"]

for year in years_:
    tr_rca[year] = tr_df[f"num_{year}"] / wo_df[f"denom_{year}"]

tr_rca

,Product code,2016,2017,2018,2019,2020,2021
0,0101,0.007452,0.008826,0.002910,0.007178,0.005520,0.004842
1,0102,0.000000,0.000000,0.005026,0.035501,0.046303,0.078439
2,0104,0.001662,0.000000,0.147827,1.405276,1.667624,1.774048
3,0105,0.890676,1.092983,1.606878,1.409566,1.394627,1.450251
4,0106,0.237821,0.247850,0.268553,0.319920,0.270732,0.249919
...,...,...,...,...,...,...,...
1224,9703,0.019845,0.008282,0.074106,0.015197,0.011358,0.032627
1225,9704,0.000000,0.000000,0.000000,0.000000,0.000000,0.017655
1226,9705,0.031887,0.025290,0.017112,0.110349,0.017151,0.007964
1227,9706,0.000000,0.000000,0.002055,0.000000,0.000000,0.000163


In [481]:
tr_rca.sort_values(by="2021")

,Product code,2016,2017,2018,2019,2020,2021
1183,9301,0.365573,0.0,0.000000,0.000000,0.0,0.0
581,5104,0.364523,0.0,0.000000,0.000000,0.0,0.0
1024,8469,0.472961,0.0,0.000000,0.000000,0.0,0.0
253,2709,0.000000,0.0,0.000000,0.000000,0.0,0.0
273,2813,0.000000,0.0,0.012215,0.000629,0.0,0.0
...,...,...,...,...,...,...,...
307,2848,0.000000,0.0,0.000000,0.000000,0.0,NaN
440,3827,NaN,NaN,NaN,NaN,NaN,NaN
1040,8485,0.000000,0.0,0.000000,0.000000,NaN,NaN
1065,8524,0.000000,0.0,0.000000,0.000000,NaN,NaN


In [482]:
tr_rca.sort_values(by="2021", ascending=False)

,Product code,2016,2017,2018,2019,2020,2021
221,2528,74.802682,75.087895,73.335243,62.546114,50.760304,49.901723
631,5511,47.605396,48.671722,47.619610,41.366318,41.585926,41.405423
647,5702,39.233814,40.433145,38.811033,39.425779,39.220087,36.900967
209,2515,43.937258,43.646994,39.033630,37.194330,36.137409,32.196666
590,5113,29.763255,32.553258,42.748119,26.488306,20.334593,23.076529
...,...,...,...,...,...,...,...
307,2848,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
440,3827,NaN,NaN,NaN,NaN,NaN,NaN
1040,8485,0.000000,0.000000,0.000000,0.000000,NaN,NaN
1065,8524,0.000000,0.000000,0.000000,0.000000,NaN,NaN


    Veriyi sıraladığımız zaman "NaN" ve "0" değerlerinin geldiğini görüyoruz. Bunları veri setinden çıkarmanın doğru bir tercih olacağını düşünüyoruz.

    Bu veriler için bir tresh hold belirleyeceğiz. Eğer ki ihracat rakamı bu tresh hold'un altında ise o değere ait bir RCA hesaplaması ilgili df'den çıkartılacaktır. 

In [483]:
t_holds = [round(tr_df[year].quantile(0.05),2) for year in years_]
t_holds

# Aşağıda görüldüğü üzere, 2021 yılına ait tresh hold 153.4'tür.

[9.0, 7.0, 9.4, 13.0, 7.0, 20.4]

In [484]:
# Eğik değerden büyük olan verileri getirelim;
tr_rca = tr_rca[tr_df["2021"]>=20.4]

In [485]:
# Yukarıdaki filtreleme ile eksik veri problemini aşamadık, hala 1 adet verimiz eksik;
tr_rca.isnull().sum()

Product code    0
2016            1
2017            0
2018            0
2019            0
2020            0
2021            0
dtype: int64

In [486]:
tr_rca[tr_rca["2016"].isnull()]["2017"]

1221    0.019571
Name: 2017, dtype: float64

In [487]:
# Eksik veriyi bir sonraki yılın verisi ile dolduralım;
tr_rca.fillna(0.019571, inplace=True)

/var/folders/l2/g3cwllyj4_139gx51n27kyn80000gn/T/ipykernel_13540/1727749342.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr_rca.fillna(0.019571, inplace=True)


In [488]:
# Eksik değerimiz kalmadı;
tr_rca.isnull().sum()

Product code    0
2016            0
2017            0
2018            0
2019            0
2020            0
2021            0
dtype: int64

In [489]:
tr_rca.shape

(1167, 7)

### Test İstatistiği Analizi

    Test İstatistiği Analizimize geçelim. Burada bağımlı repeated measures t-test kullananacağız. Testin varsayımlarını hatırlayalım;

* Bağımlı değişken sürekli olmalıdır (aralık/oran). 

* Gözlemler birbirinden bağımsızdır.
* Bağımlı değişken yaklaşık olarak normal dağılmalıdır.
* Bağımlı değişken herhangi bir aykırı değer içermemelidir.

    Verimizde bu varsayımların sağlanıp sağlanmadığına bakalım;

* Bağımlı değişken RCA süreklidir.

* Gözlemlerin bağımsızlığı genellikle test edilebilir değildir, ancak veri toplama sürecinde değişiklik yok ve rastgele ise makul bir şekilde varsayılabilir. Verimizi burada hesapladık, hesaplama süreci aynı şekilde gerçekleştiği için bağımsız olduğunu varsayacağız.
* Normallik varsayımları aşağıda kontrol edilecektir.
* Aykırı değer aşağıda kontrol edilecektir.

In [490]:
from scipy.stats import shapiro, ttest_rel
import matplotlib.pyplot as plt

In [491]:
# Aykırı değerleri kontrol edelim, var ise tıraşlayalım;

for year in years_:
    print(year, et.check_outlier(tr_rca, year, q1=0.25, q3=0.75))
    et.replace_with_thresholds(tr_rca, year, q1=0.25, q3=0.75)
    print(year, et.check_outlier(tr_rca, year, q1=0.25, q3=0.75))
    print("~~~~~~~~~~~~")

2016 True
2016 False
~~~~~~~~~~~~
2017 True
2017 False
~~~~~~~~~~~~
2018 True
2018 False
~~~~~~~~~~~~
2019 True
2019 False
~~~~~~~~~~~~
2020 True
2020 False
~~~~~~~~~~~~
2021 True
2021 False
~~~~~~~~~~~~


In [ ]:
# Normallik varsayımı;
tr_rca["diff_2017_2016"] = tr_rca["2017"] - tr_rca["2016"]
tr_rca["diff_2018_2017"] = tr_rca["2018"] - tr_rca["2017"]
tr_rca["diff_2019_2018"] = tr_rca["2019"] - tr_rca["2018"]
tr_rca["diff_2020_2019"] = tr_rca["2020"] - tr_rca["2019"]
tr_rca["diff_2021_2020"] = tr_rca["2021"] - tr_rca["2020"]

In [493]:
# Fark sütunlarını yakalayıp listeye kaydedelim;
diff_cols = [col for col in tr_rca.columns if "diff" in col]

# Shapironun hipotezlerini getirelim;
# Normallik Varsayımı

# H0: Normal dağılım varsayımı sağlanmaktadır.
# H1:..sağlanmamaktadır.

for col in diff_cols:
    test_stat, pvalue = shapiro(tr_rca[col])
    print(f'Test Stat = {round(test_stat, 4)}, p-value = {round(pvalue,4)}')

Test Stat = 0.5577, p-value = 0.0
Test Stat = 0.6083, p-value = 0.0
Test Stat = 0.641, p-value = 0.0
Test Stat = 0.6364, p-value = 0.0
Test Stat = 0.7616, p-value = 0.0


    Verilerimiz normal dağılmıyor, lakin Merkezi limit teoreminden dolayı bu varsayımın da sağlandığını varsayacağız. Zira daha iyi bir sonuç verecek olan parametrik bir test uygulamak istiyoruz.

Not: Varyans homojenliğini de paired t-test uygulayacağımız için kontrol etmeyeceğiz.

In [494]:
for year in years_:
    print(f"{year}  :", round(tr_rca[year].mean(), 4))

2016  : 0.8905
2017  : 0.8965
2018  : 0.9298
2019  : 0.9282
2020  : 0.9555
2021  : 1.0035


In [495]:
# Paired t-test hipotezini kuralım;

# H0: μ_önce = μ_sonra
# H1: μ_önce < μ_sonra

In [529]:
# 2017-2016 kıyaslaması;

test_stat, pvalue = ttest_rel(tr_rca["2016"], tr_rca["2017"], alternative="less")
print("2017-2016 kıyaslaması")
print(f'Test Stat = {round(test_stat, 4)}, p-value = {round(pvalue,4)}', end="\n\n\n")

# 2018-2017 kıyaslaması;

test_stat, pvalue = ttest_rel(tr_rca["2017"], tr_rca["2018"], alternative="less")
print("2018-2017 kıyaslaması")
print(f'Test Stat = {round(test_stat, 4)}, p-value = {round(pvalue,4)}', end="\n\n\n")

# 2019-2018 kıyaslaması;

test_stat, pvalue = ttest_rel(tr_rca["2018"], tr_rca["2019"], alternative="less")
print("2019-2018 kıyaslaması")
print(f'Test Stat = {round(test_stat, 4)}, p-value = {round(pvalue,4)}', end="\n\n\n")

# 2020-2019 kıyaslaması;

test_stat, pvalue = ttest_rel(tr_rca["2019"], tr_rca["2020"], alternative="less")
print("2020-2019 kıyaslaması")
print(f'Test Stat = {round(test_stat, 4)}, p-value = {round(pvalue,4)}', end="\n\n\n")

# 2021-2020 kıyaslaması;

test_stat, pvalue = ttest_rel(tr_rca["2020"], tr_rca["2021"], alternative="less")
print("2021-2020 kıyaslaması")
print(f'Test Stat = {round(test_stat, 4)}, p-value = {round(pvalue,4)}', end="\n\n\n")

2017-2016 kıyaslaması
Test Stat = -0.6651, p-value = 0.2531


2018-2017 kıyaslaması
Test Stat = -3.8121, p-value = 0.0001


2019-2018 kıyaslaması
Test Stat = 0.1815, p-value = 0.572


2020-2019 kıyaslaması
Test Stat = -3.2731, p-value = 0.0005


2021-2020 kıyaslaması
Test Stat = -6.0472, p-value = 0.0


